In [1]:
import sqlite3
from tabulate import tabulate
def output(query):
    '''returns output of an sql query in tabular form'''
    corr = cur.execute(query)
    output = corr.fetchall()
    header = [x[0] for x in corr.description]
    print(tabulate(output,headers = header))
con = sqlite3.connect("tutorial.db")
cur = con.cursor()
cur.execute("PRAGMA foreign_keys = 1");
cur.execute("PRAGMA table_info(students)");
cur.execute("PRAGMA table_info(courses)");
cur.execute("DROP TABLE IF EXISTS COURSES");
cur.execute("DROP TABLE IF EXISTS STUDENTS");
cur.execute('''CREATE TABLE STUDENTS(
student_id varchar(50) primary key not null, 
student_first_name varchar(50), 
student_last_name varchar(50), 
student_phone varchar(50) unique)''');
data = [('MSDS01','Jane','Doe','11111111'),\
       ('MSDS02','John','Doe','11111112'),
       ('MSDS03','Jack','Doe','11111113'),
       ('MSDS04','Joe','Doe','11111114')]
cur.executemany("INSERT INTO students VALUES(?, ?, ?, ?)", data);
cur.execute('''CREATE TABLE COURSES(
course_number varchar(50) not null,
course_name varchar(50),
student_id varchar(50) not null,
FOREIGN KEY(student_id) REFERENCES students(student_id),
primary key (course_number, student_id) )''');
data_courses = [('DSF111','Prob and Stats','MSDS01'),
               ('DSF211','Computation for analytics','MSDS01'),
               ('DSF311','Exploratory Data Analysis','MSDS01'),
               ('DSF111','Prob and Stats','MSDS02'),
               ('DSF211','Computation for analytics','MSDS02'),
               ('DSF311','Exploratory Data Analysis','MSDS03')]
cur.executemany("INSERT INTO COURSES VALUES(?,?,?)",data_courses);

# <b>Tables & Keys</b>
1. Tables are data that is stored in a row/column manner as shown below.</br>
2. Keys are attributes that uniquely identify records or a combination of records in tables. </br>
3. The table “STUDENT” consists of student information. </br>
4. while the table “STUD_COURSE” consists of the courses that a student is enrolled in.</br>

Here are the outputs of both the tables

In [2]:
output('select * from students')

student_id    student_first_name    student_last_name      student_phone
------------  --------------------  -------------------  ---------------
MSDS01        Jane                  Doe                         11111111
MSDS02        John                  Doe                         11111112
MSDS03        Jack                  Doe                         11111113
MSDS04        Joe                   Doe                         11111114


In [3]:
output('select * from courses')

course_number    course_name                student_id
---------------  -------------------------  ------------
DSF111           Prob and Stats             MSDS01
DSF211           Computation for analytics  MSDS01
DSF311           Exploratory Data Analysis  MSDS01
DSF111           Prob and Stats             MSDS02
DSF211           Computation for analytics  MSDS02
DSF311           Exploratory Data Analysis  MSDS03


## <br><b>Unique Key</b></br>
1. In RDMS, a unique key is a key that enforces uniqueness but is not the primary key. 
2. A unique key can take NULL values

In the students table, every student should ideally have a unique student phone number. 
However, student phone number is not a primary key.

In [4]:
data_unique_key = ('MSDS06','James','Doe','11111115')
cur.execute("INSERT INTO students VALUES(?, ?, ?, ?)", data_unique_key);

In [5]:
output('select * from students')

student_id    student_first_name    student_last_name      student_phone
------------  --------------------  -------------------  ---------------
MSDS01        Jane                  Doe                         11111111
MSDS02        John                  Doe                         11111112
MSDS03        Jack                  Doe                         11111113
MSDS04        Joe                   Doe                         11111114
MSDS06        James                 Doe                         11111115


In [6]:
data_unique_key_one = ('MSDS06','James','Doe',None)
cur.execute("INSERT INTO students VALUES(?, ?, ?, ?)", data_unique_key_one);
output('select * from students')

IntegrityError: UNIQUE constraint failed: STUDENTS.student_id

Unique keys can take null values.

In [7]:
data_unique_key_one = ('MSDS07','Jessica','Doe',None)
cur.execute("INSERT INTO students VALUES(?, ?, ?, ?)", data_unique_key_one);

In [8]:
output('select * from students')

student_id    student_first_name    student_last_name      student_phone
------------  --------------------  -------------------  ---------------
MSDS01        Jane                  Doe                         11111111
MSDS02        John                  Doe                         11111112
MSDS03        Jack                  Doe                         11111113
MSDS04        Joe                   Doe                         11111114
MSDS06        James                 Doe                         11111115
MSDS07        Jessica               Doe


## <br><b>Primary Key</b></br>
1. A primary key is a key that can uniquely identify every record in the table.</br>
2. In the table “STUDENT”, the primary key is the STUD_NO column. </br>
3. A student can have only 1 student number(even if two students’ names match, their student numbers are still different). </br>
4. A primary key has to be unique and not null</br>

For. e.g. a new student with a new student number can be inserted into a table.

In [9]:
data_newstudent = ('MSDS05','Jill','Doe','11111119')
cur.execute("INSERT INTO students VALUES(?, ?, ?, ?)", data_newstudent);
output('select * from students');

student_id    student_first_name    student_last_name      student_phone
------------  --------------------  -------------------  ---------------
MSDS01        Jane                  Doe                         11111111
MSDS02        John                  Doe                         11111112
MSDS03        Jack                  Doe                         11111113
MSDS04        Joe                   Doe                         11111114
MSDS06        James                 Doe                         11111115
MSDS07        Jessica               Doe
MSDS05        Jill                  Doe                         11111119


Now let us try to insert the details of the same student again. This will throw an error

In [10]:
data_newstudent_one = ('MSDS05','Jill','Doe','11111116')
cur.execute("INSERT INTO students VALUES(?, ?, ?, ?)", data_newstudent_one);

IntegrityError: UNIQUE constraint failed: STUDENTS.student_id

Also, a primary key does not accept a NULL value

In [11]:
data_nullstudent = (None,'Jessica','Doe','11111116')
cur.execute("INSERT INTO students VALUES(?, ?, ?, ?)", data_nullstudent);

IntegrityError: NOT NULL constraint failed: STUDENTS.student_id

## <br><b>Foreign Key</b></br>
A foreign key is a key that refers to the primary key in another table. 
STUD_NO in “COURSES” is a foreign key. 
Students enrolling in courses must have a student ID registered. 
Hence the foreign key in “STUDENT_COURSE” is pointing to the primary key in “STUDENT”. 
1. This ensures that there is no student number who cannot be identified in the “COURSES” table.
2. The foreign key links both tables and enforces referential integrity. 
3. This referential integrity ensures that for every foreign key, there is a corresponding table with the same attribute as a primary key. 
4. If a column is assigned a foreign key, each row of that column must contain a value that exists in the ‘foreign’ column it references.

Now let us try to insert a new value in courses as belows:-

In [ ]:
data_foreign_example_one = ('DSF111','Prob and Stats','MSDS03')
cur.execute("INSERT INTO courses VALUES(?, ?, ?)", data_foreign_example_one);
output('select * from courses')

A student who has not been registered cannot register for courses. This will create an error

In [ ]:
data_foreign_example_one = ('DSF111','Prob and Stats','MSDS10')
cur.execute("INSERT INTO courses VALUES(?, ?, ?)", data_foreign_example_one)

Since a foreign key is refering a primary key, it cannot be null as well.

In [ ]:
data_foreign_example_two = ('DSF111','Prob and Stats',None)
cur.execute("INSERT INTO courses VALUES(?, ?, ?)", data_foreign_example_two)

## <br><b>Composite Key</b></br>
1. A composite key is made by the combination of two or more columns in a table that can be used to uniquely identify each row in the table.
2. When the columns are combined uniqueness of a row is guaranteed, but when it is taken individually it does not guarantee uniqueness.
3. It can also be understood as a primary key made by the combination of two or more attributes to uniquely identify every row in a table.

In the courses table, we know that a student cannot enrol in the same course twice. However, a course can be taken
by multiple students and a student can take multiple courses.

In [ ]:
output('select * from courses')

Now let us say that Jack(Student ID:MSDS03) wants to enrol in "Computation in analytics".
It is possible because Jack has not enrolled in it before. 

In [ ]:
data_composite_example_one = ('DSF211','Computation for analytics','MSDS03')
cur.execute("INSERT INTO courses VALUES(?, ?, ?)", data_composite_example_one);
output('select * from courses')

He cannot get enrolled in Computation for analytics again as he is already enrolled. 

In [ ]:
data_composite_example_two = ('DSF211','Computation for analytics','MSDS03')
cur.execute("INSERT INTO courses VALUES(?, ?, ?)", data_composite_example_two);

Hence, the composite key of the table is course_number and student_ID. 
Both these columns can uniquely identify every row in the courses table.